In [2]:
import pandas as pd
import numpy as np
import ast


In [175]:
#load data 
load_data = pd.read_csv('../data/raw/moviesData_08-04-2025_10-07-51.csv')

#create dataframe
df = pd.DataFrame(load_data)
#df.head()

In [ ]:
drop_cols = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
df = df.drop(columns=drop_cols)
# df.head(3)

In [ ]:
# Extracting nested columns in the data
df['genres'] = df['genres'].apply(lambda x: "|".join([d['name'] for d in ast.literal_eval(x)]) if pd.notnull(x) else np.nan)

df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: ast.literal_eval(x)['name'] if pd.notnull(x) else np.nan)

df['production_countries'] = df['production_countries'].apply(lambda x: "|".join([d['name'] for d in ast.literal_eval(x)]) if pd.notnull(x) else np.nan)

df['production_companies'] = df['production_companies'].apply(lambda x: "|".join([d['name'] for d in ast.literal_eval(x)]) if pd.notnull(x) else np.nan)

df['spoken_languages'] = df['spoken_languages'].apply(lambda x: "|".join([d['english_name'] for d in ast.literal_eval(x)]) if pd.notnull(x) else np.nan)

#new columns inclusion from nested 




In [178]:
df['genres'] = df['genres'].apply(lambda x: '|'.join(sorted(x.split('|'))))
df['production_countries'] = df['production_countries'].apply(lambda x: '|'.join(sorted(x.split('|'))))
df['spoken_languages'] = df['spoken_languages'].apply(lambda x: '|'.join(sorted(x.split('|'))))

In [157]:
print(df['genres'].value_counts())
print(df['belongs_to_collection'].value_counts())
print(df['production_countries'].value_counts())
print(df['production_companies'].value_counts())
print(df['spoken_languages'].value_counts())

genres
Action|Adventure|Science Fiction             7
Action|Adventure|Science Fiction|Thriller    2
Action|Adventure|Fantasy|Science Fiction     1
Drama|Romance                                1
Adventure|Animation|Drama|Family             1
Action|Crime|Thriller                        1
Adventure|Fantasy                            1
Adventure|Animation|Comedy|Family|Fantasy    1
Adventure|Animation|Family|Fantasy           1
Family|Fantasy|Romance                       1
Action|Adventure|Animation|Family            1
Name: count, dtype: int64
belongs_to_collection
The Avengers Collection                4
Star Wars Collection                   2
Frozen Collection                      2
Jurassic Park Collection               2
Avatar Collection                      1
The Lion King (Reboot) Collection      1
The Fast and the Furious Collection    1
Black Panther Collection               1
Harry Potter Collection                1
The Incredibles Collection             1
Name: count, dtype

In [160]:
# Convert budget, id, popularity to numeric
df['budget'] = pd.to_numeric(df['budget'],errors='coerce')
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')
df['id'] = pd.to_numeric(df['id'],errors='coerce')
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df.head(1)

,backdrop_path,belongs_to_collection,budget,genres,id,origin_country,original_language,overview,popularity,poster_path,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,credits
0,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg,The Avengers Collection,356000000,Action|Adventure|Science Fiction,299534,['US'],en,After the devastating events of Avengers: Infi...,22.6136,/ulzhLuWrPK07P1YkdWQLZnQh1JL.jpg,...,2019-04-24,2799439100,181,English|Japanese|Xhosa,Released,Avenge the fallen.,Avengers: Endgame,8.238,26200,"{'cast': [{'adult': False, 'gender': 2, 'id': ..."


In [163]:
df[['budget', 'revenue', 'runtime']] = df[['budget', 'revenue', 'runtime']].replace(0, np.nan)